## ORACLE Set-up and Evaluation

In [1]:
### Data Processing

## Ignore warnings
import warnings
warnings.filterwarnings("ignore")
# Helper imports
import numpy as np
import pandas as pd
import os
import sys

from functools import reduce
import matplotlib.pyplot as plt

%matplotlib inline

current_working_dir = os.getcwd()
print(f"Current Working Directory: {current_working_dir}")
project_root = os.path.dirname(current_working_dir)
modules_path = os.path.join(project_root, 'Modules')
if modules_path not in sys.path:
    sys.path.append(modules_path)
    print(f"Added to sys.path for custom modules: {modules_path}")

####################################################################
#### NYSE Daily Open-Close Returns
####################################################################
data_folder_path = os.path.join(project_root, 'Data')
data_file_name = "OPCL_20000103_20201231.csv"   
data_file_path = os.path.join(data_folder_path, data_file_name) # So that we get to the file itself and not the folder it is in
returns_df = pd.read_csv(data_file_path) # Assumes file exists and is readable
returns_df.set_index('ticker', inplace=True)
returns_df.columns = pd.to_datetime(returns_df.columns.str.lstrip('X'), format='%Y%m%d').strftime('%Y-%m-%d')
returns_df_cleaned = returns_df.dropna().transpose() # Assumes dropna results in non-empty returns_df
returns_df_cleaned.index = pd.to_datetime(returns_df_cleaned.index)
returns_df_cleaned.index.name = 'date'
print("Data loaded and cleaned. Sample (first 5 rows/cols):")
print(returns_df_cleaned.iloc[0:5,0:5])
print(f"Shape of the cleaned data: {returns_df_cleaned.shape}")


Current Working Directory: c:\Users\hktan\OneDrive\Codes\ICAIF_25\Current_Code\Script
Added to sys.path for custom modules: c:\Users\hktan\OneDrive\Codes\ICAIF_25\Current_Code\Modules
Data loaded and cleaned. Sample (first 5 rows/cols):
ticker            AA       ABM       ABT       ADI       ADM
date                                                        
2000-01-03 -0.013042 -0.009188 -0.007117 -0.036071  0.000000
2000-01-04  0.010043  0.012346 -0.012786 -0.044261  0.005277
2000-01-05  0.047628 -0.006192  0.011111  0.014493 -0.015915
2000-01-06 -0.011713  0.000000  0.032553 -0.027719  0.010695
2000-01-07 -0.016118  0.003091  0.028573  0.033654  0.005249
Shape of the cleaned data: (5279, 663)


### Confounding Data

In [2]:
####################################################################
#### Constructing the dataframe for the Confounding Variables
####################################################################

"""
Data Source: Federal Reserve Economic Data (FRED)

DFF -> Federal Funds Effective Rate
T5YIE -> 5-Year Breakeven Inflation Rate (only from 2003)
USEPUINDXD -> Economic Policy Uncertainty Index for United States; News-based, 7 days moving average
VIX -> CBOE Volatility Index (VIX)
DCOILWTICO -> West Texas Intermediate (WTI) Crude Oil Prices: Cushing, Oklahoma
DTWEXBGS -> Broad U.S. Dollar Index: Trade Weighted Exchange Rate Index for Major Currencies (only from 2006)
DTWEXEMEGS-> Broad U.S. Dollar Index: Trade Weighted Exchange Rate Index for Emerging Market Economies (only from 2006)

Other Potential Data Sources:
Gold Prices (Futures/Spot), etc.
"""

files = ["DFF_20000103_20201231.csv",
         "T5YIE_20030102_20201231.csv",
         "USEPUINDXD_20000103_20201231.csv",
         "VIX_20000103_20201231.csv",
         "DCOILWTICO_20000103_20201231.csv",
         "DTWEXBGS_20060102_20201231.csv",
         "DTWEXEMEGS_20060102_20201231.csv"
         ]


merged_confound_df = reduce(
    lambda left, right: pd.merge(left, right, on="observation_date", how="outer"),
    (pd.read_csv(os.path.join(data_folder_path, f), parse_dates=[0]) for f in files)
)
imputed_confound_df = merged_confound_df.interpolate(method='linear', limit_direction='both')
imputed_confound_df.set_index('observation_date', inplace=True)
imputed_confound_df.index.name = 'date'  # Renaming index to 'date'
print("Confounding Variables DataFrame constructed and cleaned. Sample (first 5 rows/cols):")
print(imputed_confound_df.iloc[0:5,:])
print(f"Shape of the cleaned confounding variables data: {imputed_confound_df.shape}")
filtered_confound_df = imputed_confound_df[imputed_confound_df.index.isin(returns_df_cleaned.index)]
print("\n")
print("Filtering by trading dates ...")
print("\n")
print("Filtered Confounding Variables DataFrame:")
print(filtered_confound_df.head())
print(f"Shape of the filtered confounding variables data: {filtered_confound_df.shape}")

ValueError: You are trying to merge on datetime64[ns] and object columns for key 'observation_date'. If you wish to proceed you should use pd.concat

## ORACLE-VAR

In [ ]:
from DML_tools import *

lookback_size = 252*4
returns_df_cleaned_prep = returns_df_cleaned.iloc[:,:20]
#returns_df_cleaned_prep = returns_df_cleaned.iloc[-1020:,:20]
filtered_confound_df_prep = filtered_confound_df.loc[returns_df_cleaned_prep.index]
print(filtered_confound_df_prep.head())
print(filtered_confound_df_prep.shape)

             DFF  T5YIE  USEPUINDXD  VIXCLS  DCOILWTICO  DTWEXBGS  DTWEXEMEGS
date                                                                         
2015-11-02  0.12   1.23       32.86   14.15       46.12  110.2188    112.4965
2015-11-03  0.12   1.27       72.75   14.54       47.88  110.2440    112.1118
2015-11-04  0.12   1.26       45.24   15.51       46.32  110.7139    112.5096
2015-11-05  0.12   1.23       33.09   15.05       45.27  110.8302    112.6224
2015-11-06  0.12   1.25       78.78   14.33       44.32  111.7603    113.2503
(1300, 7)


In [ ]:
import time

# Record start time
start_time = time.perf_counter()

results = parallel_rolling_window_OR_VAR_w_para_search(asset_df = returns_df_cleaned_prep, confound_df = filtered_confound_df_prep,
                                                       days_valid=2, lookback_days = lookback_size, max_threads=8)

# Calculate and display execution time
end_time = time.perf_counter()
execution_time = end_time - start_time

print(f"rolling_window_OR_VAR() execution time: {execution_time:.2f} seconds")
print(f"rolling_window_OR_VAR() execution time: {execution_time/60:.2f} minutes")

# Store timing results for later analysis
results['execution_time_seconds'] = execution_time


It is day 1008 out of 1299 days in the dataset.
Validation errors for different p values: [(1, 0.020472118439553114), (2, 0.021008564411497392), (3, 0.021503215523249852), (4, 0.021417372805234747), (5, 0.020899126522255698)]
It is day 1009 out of 1299 days in the dataset.
Validation errors for different p values: [(1, 0.018757393408902173), (2, 0.018671704186738695), (3, 0.018303539157304295), (4, 0.018802496783058335), (5, 0.01998208457118298)]
It is day 1010 out of 1299 days in the dataset.
Validation errors for different p values: [(1, 0.017113286915052844), (2, 0.015730401832793173), (3, 0.016537321451512085), (4, 0.016621185639129718), (5, 0.018533492310155456)]
It is day 1011 out of 1299 days in the dataset.
Validation errors for different p values: [(1, 0.017331948529782856), (2, 0.01684689929456959), (3, 0.016911589961029254), (4, 0.01620282484601791), (5, 0.017200134975201183)]
It is day 1012 out of 1299 days in the dataset.
Validation errors for different p values: [(1, 0.01

In [ ]:
import pickle

# Save the dictionary
with open('result_full_orvar.pkl', 'wb') as f:
    pickle.dump(results, f)

# Load the dictionary
with open('result_full_orvar.pkl', 'rb') as f:
   results = pickle.load(f)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract p_optimal from your results dictionary
p_optimal = results['p_optimal']
days = np.arange(len(p_optimal))

# Create the plot
plt.figure(figsize=(15, 6))
plt.bar(days, p_optimal)
plt.xlabel('Day')
plt.ylabel('p_optimal Value')
plt.title('p_optimal Values Over Time')
plt.grid(True, alpha=0.3)
plt.show()

forecasted_returns = pd.DataFrame(results['Y_hat_next_store'],
                                  index=returns_df_cleaned_prep.index[lookback_size+1:],
                                  columns=returns_df_cleaned_prep.columns)
actual_returns = returns_df_cleaned_prep.iloc[lookback_size+1:,:20]
print(forecasted_returns.shape)
print(actual_returns.shape)

pnl_series_n, pnl_per_n = calculate_pnl(forecasted_returns, actual_returns, contrarian = False, pnl_strategy="naive")
pnl_series_w, pnl_per_w = calculate_pnl(forecasted_returns, actual_returns, contrarian = False, pnl_strategy="weighted")
pnl_series_t, pnl_per_t = calculate_pnl(forecasted_returns, actual_returns, contrarian = False, pnl_strategy="top")

'''
Example
forecasted_returns = [[3,4],
                      [2,1],
                      [5,6]]
Each column is an asset, and each row is a day (prediction for next three days, for 2 assets)

actual_returns     = [[4,4],
                      [5,1],
                      [5,7]]

calculate_pnl returns a a series of PnL values for each day based on the strategy specified;

shape = [[1.04],
         [0.978],
         [1.01]]
    where each value is the PnL for that day based on the strategy (these are cumulative PnL)

'''

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(pnl_series_n, color='blue', label='Naive Strategy', linewidth=2)
plt.plot(pnl_series_w, color='red', label='Weighted Strategy', linewidth=2)
plt.plot(pnl_series_t, color='green', label='Top Strategy', linewidth=2)

# Customize the plot
plt.title('PnL Comparison Across Different Strategies', fontsize=16)
plt.xlabel('Time', fontsize=12)
plt.ylabel('PnL', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()

plt.savefig('pnl_comparison.png')
plt.show()

def get_metrics(series, perc):
    # For a compounding PnL series, daily returns are the percentage change.
    daily_rets = perc.dropna()

    # Calculate Compound Annual Growth Rate (CAGR)
    total_return = series.iloc[-1] 
    ann_ret = (1 + total_return) ** (252 / len(series)) - 1

    # Sharpe ratio calculation remains valid with the corrected daily returns.
    # We assume a risk-free rate of 0 for this calculation.
    sharpe = (daily_rets.mean() * 252) / (daily_rets.std() * np.sqrt(252))
    return ann_ret * 100, sharpe

# Get metrics for all strategies
for name, series, perc in [("Naive", pnl_series_n, pnl_per_n), ("Weighted", pnl_series_w, pnl_per_w), ("Top", pnl_series_t, pnl_per_t)]:
    ann_ret, sharpe = get_metrics(series, perc)
    print(f"{name}: {ann_ret:.4f}% annualized, Sharpe: {sharpe:.4f}")

(291, 20)
(291, 20)
Daily portfolio returns (percentage change): date
2019-11-06    0.000545
2019-11-07   -0.004616
2019-11-08    0.000764
2019-11-11   -0.000472
2019-11-12    0.001946
                ...   
2020-12-24    0.000182
2020-12-28   -0.000423
2020-12-29    0.000691
2020-12-30    0.004419
2020-12-31    0.006377
Length: 291, dtype: float64
Daily portfolio returns (percentage change): date
2019-11-06    0.003940
2019-11-07   -0.001649
2019-11-08    0.000936
2019-11-11    0.000023
2019-11-12    0.003721
                ...   
2020-12-24   -0.003320
2020-12-28    0.000512
2020-12-29    0.002614
2020-12-30    0.007299
2020-12-31    0.009000
Length: 291, dtype: float64
Daily portfolio returns (percentage change): date
2019-11-06    0.000000
2019-11-07    0.000000
2019-11-08    0.000000
2019-11-11    0.003930
2019-11-12    0.000000
                ...   
2020-12-24    0.008470
2020-12-28    0.016000
2020-12-29    0.000000
2020-12-30    0.000000
2020-12-31    0.012203
Length: 291, dt

'\nExample\nforecasted_returns = [[3,4],\n                      [2,1],\n                      [5,6]]\nEach column is an asset, and each row is a day (prediction for next three days, for 2 assets)\n\nactual_returns     = [[4,4],\n                      [5,1],\n                      [5,7]]\n\ncalculate_pnl returns a a series of PnL values for each day based on the strategy specified;\n\nshape = [[1.04],\n         [0.978],\n         [1.01]]\n    where each value is the PnL for that day based on the strategy (these are cumulative PnL)\n\n'

# Parallelized ORACLE-VAR

In [ ]:
from DML_parallelized import *
import time

returns_df_cleaned_prep = returns_df_cleaned.iloc[:,:20]
filtered_confound_df_prep = filtered_confound_df.loc[returns_df_cleaned_prep.index]
print(filtered_confound_df_prep.head())
print(filtered_confound_df_prep.shape)

             DFF  T5YIE  USEPUINDXD  VIXCLS  DCOILWTICO  DTWEXBGS  DTWEXEMEGS
date                                                                         
2016-12-12  0.41   1.86       99.45   12.64       52.74  116.9161    122.3953
2016-12-13  0.41   1.86      336.71   12.72       52.99  116.8029    122.3827
2016-12-14  0.41   1.82       53.87   13.19       51.01  116.7332    122.3870
2016-12-15  0.66   1.78      232.94   12.79       50.90  118.8072    123.7085
2016-12-16  0.66   1.75       53.13   12.20       51.93  118.3584    123.4953
(1020, 7)


In [ ]:
# Record start time
start_time = time.perf_counter()

results = parallel_rolling_window_ORACLE_VAR(
    asset_df=returns_df_cleaned_prep,
    confound_df=filtered_confound_df_prep,
    p_max=10, max_threads=2, lookback_days = lookback_size)

# results = parallel_rolling_window_OR_VAR(asset_df = returns_df_cleaned_prep, confound_df = filtered_confound_df_prep,
#                                         days_valid = 2, max_threads=2)

# Calculate and display execution time
end_time = time.perf_counter()
execution_time = end_time - start_time

print(f"rolling_window_ORACLE_VAR() execution time: {execution_time:.2f} seconds")
print(f"rolling_window_ORACLE_VAR() execution time: {execution_time/60:.2f} minutes")

# Store timing results for later analysis
results['execution_time_seconds'] = execution_time

Start time: 2025-07-02 18:00:57
Beginning search for optimal VAR order for each day
Per-day optimal VAR order (p) and corresponding validation error:
[[1008.0, 1.0, 1.0, 0.03425226527863388], [1009.0, 4.0, 1.0, 0.044289888409094244], [1010.0, 5.0, 1.0, 0.04168681283096512], [1011.0, 1.0, 1.0, 0.029561173250564052], [1012.0, 1.0, 1.0, 0.024450859837424926], [1013.0, 1.0, 1.0, 0.03117996954671924], [1014.0, 1.0, 1.0, 0.031061574327516155], [1015.0, 1.0, 1.0, 0.024933811204249607], [1016.0, 1.0, 1.0, 0.01770010144683326], [1017.0, 1.0, 1.0, 0.016229376878262564], [1018.0, 4.0, 1.0, 0.018562025888797108]]
Completed VAR order search
Elapsed time: 137.5327 seconds
Computing daily predictions using the observed optimal VAR order
Completed predictions
End time: 2025-07-02 18:03:34
Total elapsed time: 157.1651 seconds
rolling_window_ORACLE_VAR() execution time: 157.17 seconds
rolling_window_ORACLE_VAR() execution time: 2.62 minutes


In [ ]:
import pickle

# Save the dictionary
with open('result_full_oraclevar.pkl', 'wb') as f:
    pickle.dump(results, f)

# Load the dictionary
with open('result_full_oraclevar.pkl', 'rb') as f:
   results = pickle.load(f)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract p_optimal from your results dictionary
p_optimal = results['p_optimal']
days = np.arange(len(p_optimal))

# Create the plot
plt.figure(figsize=(15, 6))
plt.bar(days, p_optimal)
plt.xlabel('Day')
plt.ylabel('p_optimal Value')
plt.title('p_optimal Values Over Time')
plt.grid(True, alpha=0.3)
plt.show()

forecasted_returns = pd.DataFrame(results['Y_hat_next_store'],
                                  index=returns_df_cleaned_prep.index[lookback_size+1:],
                                  columns=returns_df_cleaned_prep.columns)
actual_returns = returns_df_cleaned_prep.iloc[lookback_size+1:,:20]
print(forecasted_returns.shape)
print(actual_returns.shape)

pnl_series_n, pnl_per_n = calculate_pnl(forecasted_returns, actual_returns, contrarian = False, pnl_strategy="naive")
pnl_series_w, pnl_per_w = calculate_pnl(forecasted_returns, actual_returns, contrarian = False, pnl_strategy="weighted")
pnl_series_t, pnl_per_t = calculate_pnl(forecasted_returns, actual_returns, contrarian = False, pnl_strategy="top")

'''
Example
forecasted_returns = [[3,4],
                      [2,1],
                      [5,6]]
Each column is an asset, and each row is a day (prediction for next three days, for 2 assets)

actual_returns     = [[4,4],
                      [5,1],
                      [5,7]]

calculate_pnl returns a a series of PnL values for each day based on the strategy specified;

shape = [[1.04],
         [0.978],
         [1.01]]
    where each value is the PnL for that day based on the strategy (these are cumulative PnL)

'''

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(pnl_series_n, color='blue', label='Naive Strategy', linewidth=2)
plt.plot(pnl_series_w, color='red', label='Weighted Strategy', linewidth=2)
plt.plot(pnl_series_t, color='green', label='Top Strategy', linewidth=2)

# Customize the plot
plt.title('PnL Comparison Across Different Strategies', fontsize=16)
plt.xlabel('Time', fontsize=12)
plt.ylabel('PnL', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()

plt.savefig('pnl_comparison.png')
plt.show()

def get_metrics(series, perc):
    # For a compounding PnL series, daily returns are the percentage change.
    daily_rets = perc.dropna()

    # Calculate Compound Annual Growth Rate (CAGR)
    total_return = series.iloc[-1] 
    ann_ret = (1 + total_return) ** (252 / len(series)) - 1

    # Sharpe ratio calculation remains valid with the corrected daily returns.
    # We assume a risk-free rate of 0 for this calculation.
    sharpe = (daily_rets.mean() * 252) / (daily_rets.std() * np.sqrt(252))
    return ann_ret * 100, sharpe

# Get metrics for all strategies
for name, series, perc in [("Naive", pnl_series_n, pnl_per_n), ("Weighted", pnl_series_w, pnl_per_w), ("Top", pnl_series_t, pnl_per_t)]:
    ann_ret, sharpe = get_metrics(series, perc)
    print(f"{name}: {ann_ret:.4f}% annualized, Sharpe: {sharpe:.4f}")